----
Qu implementation second half
====



The steps here are somewhat straightforward:
- We take the output of a network, get the control points from it
- We generate a voronoi pattern from this, separating the image into sections where we are positive there is only one nucleus.
- We use k-means clustering for all pixels, using distance data from the nucleus and using color data to arrive at a segmentation that envelops the nucleus.

In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob

from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy.ndimage import binary_fill_holes

from mask_prediction import start_over as qu
from mask_prediction import unet_semantics as model_setup

def watershed(img, dist_thresh_scale=.4):
    """
    This algorithm performs a form of watershed operation. After some morphological operations,
    a distance transform with a threshold will separate cleanly all blobs that would have been too close to do a contour analysis.
    :param img: The image to be watershedded.
    :param dist_thresh_scale: Ratio of where to put the threshold of the watershedding.
    :return: A sure foreground image, alongside an unsure image. The highlighted pixels in unsure could belong to the foreground or the background.
    """
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=1)
    sure_bg = cv2.dilate(opening, kernel, iterations=1)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, dist_thresh_scale*dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    return sure_fg, unknown

In [6]:
IMG_HEIGHT = 1024
IMG_WIDTH = 1024

nucl_rad = 75

run_name = 'zebra_zoom2_test'
dataset = 'RL015'
ini_data_path = 'X:\\BEP_data\\Data_Internal\\Annotation_Iteration'                         #File containing data structure
em_folder = 'X:\\BEP_data\\Data_External\\{}\\EM'.format(dataset)       #Folder containing the EM datasets
ho_folder = 'X:\\BEP_data\\Data_External\\{}\\Hoechst'.format(dataset)  #Folder containing the Hoechst datasets
predict_folder = 'X:\\BEP_data\\Annotation_Iteration\\Predict_Backups\\qu_zebra_test_2zoom_emfm_254_2021-10-06_21-10-07\\Output'
export_folder = 'X:\\BEP_data\\Data_Internal\\Annotation_Iteration\\Generated_set'
train_folder = []                                        #Because this notebook does not use Machine Learning, the training and testing folders are not populated.
test_folder = []
mask_folder = 'X:\\BEP_data\\Data_External\\{}\\PPA_Upscale\\'.format(dataset)
nr_clusters = 4
fill_holes = True

data_paths = (train_folder, test_folder, em_folder, ho_folder, mask_folder)

The parameters are set, time to import the masks, and get a list of nuclei positions:

In [7]:
mask_list = glob(predict_folder + '\\*.png')
str_list = [x.split('\\')[-1] for x in mask_list]

nuclei_dict = {}

"""
For every mask in the mask list, the mask is thresholded,
watershedded and its countours are analyzed to get at an accurate list of nuclei positions.

TODO: The watershedding might be overkill.
"""
for mask in mask_list:
    img = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    img_str = mask.split('\\')[-1]
    _, img_thresh = cv2.threshold(img, int(255*.7), 255, cv2.THRESH_BINARY)

    img_wet, unknown = watershed(img_thresh)
    cnts, _ = cv2.findContours(img_wet, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    ncls_pts = []
    for cnt in cnts:
        if cv2.contourArea(cnt) >= 1:
            M = cv2.moments(cnt)
            coords = [int(M['m10']/M['m00']), int(M['m01']/M['m00'])]
            ncls_pts.append(coords)
    nuclei_dict[img_str] = ncls_pts



So now we have a list for every image that we are processing of where the network thought the nuclei are. Now comes the real work, using the positions to generate a voronoi pattern.
This pattern will break up the image into segments that contain exactly one nucleus. In the next piece of code, the pattern is calculated,
and the data that will be used in k-means clustering is sandwiched. This data will consist of the EM data, the Hoechst data and a distance map based on the average diameter of the nuclei.

In [8]:
"""
In order to facilitate the distance threshold later, a distance limit map is made,
filled with the square of the diameter of an average nucleus.

Mesh grids are also generated to aid many other operations down the line. One prominent is the rescaling of 2d arrays
into 1d arrays. Having a mesh grid in that rescaling will keep track of where the pixel belongs in the original image.
"""

num_range = np.arange(0, 1024, 1, dtype=np.int32)
dist_limit = nucl_rad * nucl_rad
dist_limit_map = np.ones((IMG_WIDTH, IMG_HEIGHT), np.int32) * dist_limit

x_meshgrid, y_meshgrid = np.meshgrid(num_range, num_range)

for image in os.listdir(export_folder + '\\Output'):
    os.remove(export_folder + '\\Output\\' + image)

for key in nuclei_dict:
    print('Currently doing {}'.format(key))


    """
    This block of code generates the lines for a Voronoi pattern. The partitions of which will be used later.
    """
    div2d = cv2.Subdiv2D()
    div2d.initDelaunay((0,0,IMG_WIDTH, IMG_HEIGHT))
    div2d.insert(nuclei_dict[key])
    vor_list, pnts = div2d.getVoronoiFacetList([])

    """
    This block of code generates a distance map for the current image. This is done here, because there are not any
    freely avaliable distance map algorithms to take advantage of.
    """
    dist_map_sq_tot = dist_limit_map
    for pnt in nuclei_dict[key]:
        x_meshgrid_s = np.abs(x_meshgrid - pnt[0])
        y_meshgrid_s = np.abs(y_meshgrid - pnt[1])
        x_meshgrid_s = np.square(x_meshgrid_s)
        y_meshgrid_s = np.square(y_meshgrid_s)
        dist_map_sq = x_meshgrid_s + y_meshgrid_s
        dist_map_sq = np.minimum(dist_map_sq, dist_limit_map)
        dist_map_sq_tot = np.minimum(dist_map_sq, dist_map_sq_tot)
    dist_map = np.sqrt(dist_map_sq_tot)/nucl_rad
    dist_map_uint = np.array(dist_map*255, np.uint8)

    """
    The next block of code generates various filters from the EM data, and imports the HO data as well.
    """
    em_img = cv2.imread(em_folder + '\\Collected\\' + key, cv2.IMREAD_GRAYSCALE)
    em_bil_img = cv2.bilateralFilter(em_img, 7 , 75, 75)
    em_gauss_img = cv2.GaussianBlur(em_img, (3,3), 3)
    ho_img = cv2.imread(ho_folder + '\\Collected\\' + key, cv2.IMREAD_GRAYSCALE)
    lap_img = cv2.Laplacian(em_gauss_img, cv2.CV_8U)
    lap_img = cv2.normalize(lap_img,None, 255, 0, cv2.NORM_MINMAX)
    em_entr_img = entropy(em_gauss_img, disk(7))
    em_entr_img  = (em_entr_img*255).astype(np.uint8)
    em_entr_img = cv2.normalize(em_entr_img, None, 255, 0, cv2.NORM_MINMAX)

    em_sobel_x_img = cv2.Sobel(em_gauss_img, cv2.CV_16S, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
    em_sobel_y_img = cv2.Sobel(em_gauss_img, cv2.CV_16S, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)

    em_sobel_x_abs_img = cv2.convertScaleAbs(em_sobel_x_img)
    em_sobel_y_abs_img = cv2.convertScaleAbs(em_sobel_y_img)

    em_sobel_img = cv2.addWeighted(em_sobel_x_abs_img, .5, em_sobel_y_abs_img, .5, 0)

    """
    Now that all the filters have been generated for the current image, they are assembled into a giant sandwich.
    The contents of the sandwich can be changed from run to run, and it is made as accessible as possible.
    Just make sure to adjust the second number in the reshape function to the amount of filters that are put into the sandwich.
    The x_meshgrid and the y_meshgrid do need to stay in the sandwich, they allow for selection between different pixels
    belonging to different voronoi partitions.
    """
    sandwich = np.dstack((y_meshgrid, x_meshgrid, dist_map_uint, em_gauss_img, em_sobel_img, ho_img))
    sandwich_r = np.reshape(sandwich, (-1, 6))

    label_map = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype=np.uint8)
    em_show = em_img
    for facet in tqdm(vor_list):
        facet_uint = np.array(facet, np.int32)
        mask = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype=np.uint8)
        mask = cv2.drawContours(mask, [facet_uint], -1, 255, -1, cv2.LINE_8)
        em_show = cv2.drawContours(em_show, [facet_uint], -1, 255, 2)
        mask_bool = mask == 255
        mask_bool_r = np.reshape(mask_bool, -1)

        flist = sandwich_r[mask_bool_r]
        labels = qu.color_k_means(flist, cluster_nr=nr_clusters)*(int(255/nr_clusters))
        label_map += labels

    label_floodfill = qu.get_floodfill(label_map, nuclei_dict[key], margin=2)

    img_EM_clustered_floodfill = np.where(label_floodfill == 0, 255, 0)
    img_EM_clustered_floodfill = (img_EM_clustered_floodfill).astype(np.uint8)

    if fill_holes:
        img_EM_clustered_floodfill = binary_fill_holes(img_EM_clustered_floodfill/255)
        img_EM_clustered_floodfill = img_EM_clustered_floodfill.astype(np.uint8)*255

    cv2.imwrite(export_folder + '\\Output\\' + key, img_EM_clustered_floodfill)
model_setup.backup_data(data_paths, '*.png', run_name, export_folder, ini_data_path + '\\Generated_backups', img_strs=str_list)

print('All done!')

Currently doing 1_10_10_2.png


100%|██████████| 29/29 [00:06<00:00,  4.33it/s]


Currently doing 1_10_11_2.png


100%|██████████| 9/9 [00:06<00:00,  1.39it/s]


Currently doing 1_10_12_2.png


...

Currently doing 5_9_8_2.png


100%|██████████| 5/5 [00:05<00:00,  1.13s/it]


Currently doing 5_9_9_2.png


100%|██████████| 53/53 [00:06<00:00,  7.86it/s]


All done!
